<a href="https://colab.research.google.com/github/GorRatsy/my-first-ex/blob/main/devatios_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sber_data = pd.read_csv('https://lms.skillfactory.ru/assets/courseware/v1/fbb2e41c55afd1333f7f87e91e904e9f/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/sber_data.zip', compression='zip', sep = ',')


In [ ]:
# graphic method
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=sber_data, x='full_sq', ax=axes[0]);
histplot.set_title('Full Square Distribution')
boxplot = sns.boxplot(data=sber_data, x='full_sq', ax=axes[1]);
boxplot.set_title('Full Square Boxplot');

In [20]:
# iqr-method

def outliers_iqr_mod(data, feature, left = 1.5, right = 1.5, log_scale = False):
    """
    Давайте немного модифицируем нашу функцию outliers_iqr(). 
    Добавьте в нее параметры left и right, которые задают число IQR влево и вправо от границ ящика (пусть по умолчанию они равны 1.5).
    Функция, как и раньше должна возвращать потенциальные выбросы и очищенный DataFrame.
    """
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_iqr_mod(sber_data, 'price_doc', left = 3, right = 3, log_scale = True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу z-отклонения: 92
Результирующее число записей: 30379


In [ ]:
# z-method to delete deviations in data

def outliers_z_score(data, feature, log_scale=False, left = 3, right = 3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(sber_data, 'price_doc', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')



In [ ]:

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (15,4))
usual_hist = sns.histplot(data = sber_data, x = 'price_doc', ax = axes[0])
usual_hist.set_title('Usual hist')
sber_data['log_price'] = np.log(sber_data['price_doc']+1)
log_hist = sns.histplot(data = sber_data, x = 'log_price', ax = axes[1])
log_hist.set_title('Log hist')
